Plots: 
- time response: pitch_dot_dot ~ 30ms
                 zero_x: ~ 85 ms
                 min_CM_dot_dot: ~ 75 ms
- linear correletion: pitch vs CM_dot_dot
- correction of velocity direction: time series: CM_x_dot_projected
                                                 delta angle between initial CM_x_dot_projected direction and final
                                    histogram :  initial angle (mean on frames) - final angle(mean on frames)
                        - memory? 
                        - correction time from light? 
                        - correct in dark? in 100 ms? not sure

- they try to keep delta yaw 0 and go back to the same velocity. its not the same target! 
they have velocity in the Y direction (slip) when they react to dark, they slip for a while and continue the according to their position 
most of them will keep the same yaw (it means they need to mesure their angular position some how and keep it the same)
- they manage to keep the same yaw while changing the velocity 
(kind of balance - they use the wings to slip but keep the yaw the same, something symetric should happen)
- they go back to the same velocity size, is it their baseline or they remember?

- when in total darkness they manage to keep the yaw but the velocity doesnt neceseely stay the same (which means it adds slip). 
* they mesure gravity - for pitch? how they keep the same yaw? (not all of them keep the same yaw, less than the pulse one) 
* velocity mesurment - oprical flow and probably antena. the fact that some of them regain the same velocity means that optical flow is not the only sensor

--- the dark response: pitch up and backwards fly  - they try to keep the same yaw and go back to the same velocity. 
if the light turns on they manage to regain the same velocity. they probably keep the same direction as a delta t from the lighs turning on

In [1]:

from Experiment import Experiment
import matplotlib.pyplot as plt
import Plotters 
import plotly.graph_objects as go
import plotly
import numpy as np

import time
import matplotlib.cm as colormap

%matplotlib qt

loadir = 'H:/My Drive/dark 2022/csv_dark' # dir to save the data frame in # H:\My Drive\Ronis Exp\sagiv\data_analysis
exp_name =['manipulated_2023_08_07_10ms',
            'manipulated_2023_08_06_40ms',
           'manipulated_2023_08_09_60ms',
           'manipulated_2023_08_09_80ms',
           'manipulated_2023_08_10_100ms',
           'manipulated_2022_02_03_dark']



def dont_show_movs(exp_name,movs_to_del):
    return [mov_name for mov_name in experiments[exp_name].mov_names if mov_name not in movs_to_del ] 

mov_dark_delete = ['mov30','mov40','mov57','mov59']
mov_100_delete = ['mov10','mov102','mov19','mov34','mov35','mov47','mov51','mov52','mov66','mov68','mov73','mov90']
mov_80_delete = ['mov119','mov49','mov97','mov99']
mov_40_delete = ['mov64']
mov_60_delete = ['mov12','mov13','mov147','mov165','mov21','mov24','mov26','mov48','mov86']
mov_10_delete = ['mov46','mov54']
mov_5_delete = ['mov44']


color_map = colormap.datad["tab10"]['listed']

experiments = {exp:Experiment(loadir,exp) for exp in exp_name} # load experiments


Smooth, calculate projected axes and angles

In [2]:
[exp.smooth_prop_movies('CM_real_x_body',['','_dot','_dot_dot'],'body')  for exp in experiments.values()] # smooth and derive


[exp.project_axes_movies(['X_x_body_projected','X_y_body_projected'])  for exp in experiments.values()] # X_body axis projected to xy plane
[exp.project_axes_movies(['Y_x_body_projected','Y_y_body_projected'],axis = 'Y_x_body')  for exp in experiments.values()] # X_body axis projected to xy plane

[exp.project_axes_movies(['CM_dot_xax','CM_dot_yax'],axis = 'CM_real_x_body_dot_smth',wing_body = 'body',add_to_vectors = True)  for exp in experiments.values()] # Velocity direction in the projected X_body axes (xy plane) 

# calculate the size of the velocity projected on the xy plane
[exp.project_prop_movies('CM_real_x_body_dot_smth',header_name = 'CM_dot_x',three_col = 2,ax_to_proj = 'X_x_body_projected',add_to_vectors = True)  for exp in experiments.values()]
[exp.project_prop_movies('CM_real_x_body_dot_smth',header_name = 'CM_dot_y',three_col = 2,ax_to_proj = 'Y_x_body_projected',add_to_vectors = True)  for exp in experiments.values()]


# calculate the size of the acceleration projected on the xy plane
[exp.project_prop_movies('CM_real_x_body_dot_dot_smth',header_name = 'CM_dot_dot_x',three_col = 2,ax_to_proj = 'X_x_body_projected')  for exp in experiments.values()]
[exp.project_prop_movies('CM_real_x_body_dot_dot_smth',header_name = 'CM_dot_dot_y',three_col = 2,ax_to_proj = 'X_y_body_projected')  for exp in experiments.values()]

[exp.pqr_movies() for exp in experiments.values()] # calculate p,q,r (angular velocity in body axis)

[exp.mean_mean_props_movies('phi_rw','phi_lw','mean_wing','mean_mean_phi') for exp in experiments.values()] # calculate the mean value of two properties


[exp.substruct_first_frame('yaw_body','body')  for exp in experiments.values()] # substract the zero frame from a property ( {prop}_min_ref_frame )

# calculate the difference between current direction and another one at the same time:
# the angle between Xbody on the xy plane and the velocity direction on the xy plane 
[exp.delta_ang_all_time_movies('X_x_body_projected','CM_dot_xax','slip_angle_beta',three_col = 2) for exp in experiments.values()] 

# calculate the difference in angle between current property and in referance frame:
# the angle between Xbody on the xy (in 0 time) plane and Xbody on xy plane 
[exp.delta_ref_ang_movies('X_x_body_projected','delta_ang_Xax',axis = 'X_x_body_projected')  for exp in experiments.values()]

# the angle between Xbody on the xy (in 0 time) plane and the velocity direction on the xy plane 
[exp.delta_ref_ang_movies('CM_dot_xax','delta_ang_CM_dot_x',axis = 'CM_dot_xax',delta_ang = 0) for exp in experiments.values()]
[exp.substruct_first_frame('delta_ang_CM_dot_x','body') for exp in experiments.values()]


c:\Users\Roni\Documents\experiment-analysis\data_analysis\Movie.py:107: RuntimeWarning:

invalid value encountered in arccos



[None, None, None, None, None, None]

In [64]:
idx_prop = 'phi_lw_max_idx'
mean_wing_body = 'mean_body'
[exp.calculate_freq_movies(idx_prop, mean_wing_body)  for exp in experiments.values()]

prop = 'CM_dot_dot_x_projected'
[exp.mean_by_stroke_movies(prop,mean_wing_body,'body')  for exp in experiments.values()]
prop = 'delta_ang_CM_dot_x'
[exp.mean_by_stroke_movies(prop,mean_wing_body,'body')  for exp in experiments.values()]
prop = 'delta_ang_Xax'
[exp.mean_by_stroke_movies(prop,mean_wing_body,'body')  for exp in experiments.values()]


c:\Users\Roni\Documents\experiment-analysis\data_analysis\Movie.py:328: RuntimeWarning:

divide by zero encountered in divide



[None, None, None, None, None, None]

In [62]:
prop = 'CM_real_z_body_dot_dot_smth'
[exp.mean_by_stroke_movies(prop,mean_wing_body,'body')  for exp in experiments.values()]

[exp.calculate_phi_amp_movies('rw',mean_wing_body)  for exp in experiments.values()]
[exp.calculate_phi_amp_movies('lw',mean_wing_body)  for exp in experiments.values()]


[None, None, None, None, None, None]

Histograms

In [3]:
exp = experiments[list(experiments.keys())[0]]
[exp.get_peaks_movies('q','max') for exp in experiments.values()]
[exp.get_peaks_movies('pitch_body','peaks_max',t0 = 40, t1 = 100) for exp in experiments.values()]
[exp.get_peaks_movies('CM_dot_dot_x_projected','min') for exp in experiments.values()]
[exp.get_peaks_movies('pitch_y_frame','max') for exp in experiments.values()]
[exp.get_peaks_movies('pitch_body','max') for exp in experiments.values()]
[exp.get_peaks_movies('CM_dot_x_projected','min') for exp in experiments.values()]
[exp.get_peaks_movies('CM_dot_x_projected','zero_v') for exp in experiments.values()]
[exp.get_peaks_movies('delta_ang_Xax','max') for exp in experiments.values()]
[exp.get_peaks_movies('CM_dot_dot_x_projected','peaks_max',t0 = 5, t1 = 100) for exp in experiments.values()]

[exp.get_peaks_movies('CM_dot_x_projected','half_v') for exp in experiments.values()]

prop  = 'CM_dot_x_projected_zero_v'
[exp.subtract_interest_time_from_time(prop,'time_m_zero_v') for exp in experiments.values()]

point_name = 'CM_dot_x_projected_zero_v'
list_trace,layout = zip(*[exp.interest_point_hist(point_name,xbins = dict(start=0,end=300, size=5)) for exp in experiments.values()])

fig = go.Figure(data=list_trace, layout=layout[0]).show()


point_name = 'CM_dot_dot_x_projected_min'
list_trace,layout = zip(*[exp.interest_point_hist(point_name,xbins = dict(start=0,end=300, size=5)) for exp in experiments.values()])
fig = go.Figure(data=list_trace, layout=layout[0]).show()


point_name = 'CM_dot_dot_x_projected_peaks_max'
list_trace,layout = zip(*[exp.interest_point_hist(point_name,xbins = dict(start=0,end=300, size=5)) for exp in experiments.values()])
fig = go.Figure(data=list_trace, layout=layout[0]).show()


In [148]:

time_fin_exp = [100,210,250,230,265,213]
delta_v = [exp.get_delta_angle_movies('delta_ang_CM_dot_x', time_prop = 'time', t_fin = tfin, delta_frames = 300) for exp,tfin in zip(experiments.values(),time_fin_exp)]

list_trace,layout = zip(*[Plotters.histogram(delt_v_exp,exp_name,'angle', 'delta_angle_velocity',xbins = dict(start=0,end=500, size=10),histnorm = None) for delt_v_exp,exp_name in zip(delta_v,experiments.keys())])
fig = go.Figure(data=list_trace, layout=layout[0]).show()


In [ ]:
exp = experiments['manipulated_2022_02_03_dark']#['manipulated_2022_02_03_dark']#['manipulated_2023_08_09_60ms']
# exp_name =['manipulated_2023_08_07_10ms',
#             'manipulated_2023_08_06_40ms',
#            'manipulated_2023_08_09_60ms',
#            'manipulated_2023_08_09_80ms',
#            'manipulated_2023_08_10_100ms',
#            'manipulated_2022_02_03_dark']

prop = 'delta_ang_CM_dot_x'
fig = go.Figure()
exp.plot_prop_movies(prop,'body',fig,case = 'plot_mov',prop_x = 'time')
fig.show()


prop = 'delta_ang_Xax'
fig = go.Figure()
exp.plot_prop_movies(prop,'body',fig,case = 'plot_mov',prop_x = 'time')
fig.show()



# prop = 'roll_body'
# fig = go.Figure()
# exp.plot_prop_movies(prop,'body',fig,case = 'plot_mov',prop_x = 'time')
# exp.plot_prop_movies('pitch_body','body',fig,case = 'plot_mov',prop_x = 'time')
# exp.plot_prop_movies('yaw_body','body',fig,case = 'plot_mov',prop_x = 'time')

# fig.show()



In [ ]:
exp = experiments['manipulated_2023_08_09_60ms']#['manipulated_2022_02_03_dark']#['manipulated_2023_08_09_60ms']['manipulated_2023_08_07_10ms','manipulated_2023_08_06_40ms',

prop = 'delta_ang_CM_dot_x'
fig = go.Figure()
exp.plot_prop_movies(prop,'body',fig,case = 'plot_mov',prop_x = 'time')
exp.plot_prop_movies('delta_ang_Xax','body',fig,case = 'plot_mov',prop_x = 'time')

fig.show()


In [63]:

fig = go.Figure()

# exp.plot_prop_movies('freq_max_rw','mean_wing',fig,case = 'plot_mov',prop_x = 'time' ,add_horizontal_line=None)
exp.plot_prop_movies('CM_real_z_body_dot_dot_smth','mean_body',fig,case = 'plot_mov',prop_x = 'time' ,add_horizontal_line=None)

fig.show()

In [75]:
exp = experiments['manipulated_2023_08_09_60ms']#['manipulated_2022_02_03_dark']#['manipulated_2023_08_09_60ms']

fig = go.Figure()

exp.plot_prop_movies('freq_phi_rw_min_idx','mean_body',fig,case = 'plot_mov',prop_x = 'phi_rw_min_val' ,
                     add_horizontal_line=None,mode = 'markers')

exp.plot_prop_movies('freq_phi_lw_max_idx','mean_body',fig,case = 'plot_mov',prop_x = 'phi_lw_max_val' ,
                     add_horizontal_line=None,mode = 'markers')

fig.show()


phi_r_color = [[256,0,0]]*len(exp.mov_names)
phi_l_color = [[0,0,256]]*len(exp.mov_names)
fig = go.Figure()
exp.plot_prop_movies('amp_rw','mean_body',fig,case = 'plot_mov',prop_x = 'phi_rw_min_val' ,
                     add_horizontal_line=None,mode = 'markers', color = phi_r_color)

exp.plot_prop_movies('amp_lw','mean_body',fig,case = 'plot_mov',prop_x = 'phi_lw_max_val' ,
                     add_horizontal_line=None,mode = 'markers',color = phi_l_color)
fig.show()

In [ ]:
exp = experiments['manipulated_2023_08_09_60ms']#['manipulated_2022_02_03_dark']#['manipulated_2023_08_09_60ms']

fig = go.Figure()

phi_r_color = [[256,0,0]]*len(exp.mov_names)
phi_l_color = [[0,0,256]]*len(exp.mov_names)
# exp.plot_prop_movies('amp_real2','mean_wing',fig,case = 'plot_mov',prop_x = 'time' ,add_horizontal_line=None)
exp.plot_prop_movies('delta_ang_CM_dot_x','body',fig,case = 'plot_mov',prop_x = 'time' ,add_horizontal_line=None)
fig.show()



fig = go.Figure()

exp.plot_prop_movies('freq','mean_wing',fig,case = 'plot_mov',prop_x = 'time' ,add_horizontal_line=None)
fig.show()
fig = go.Figure()

exp.plot_prop_movies('phi_rw_min_val','mean_wing',fig,case = 'plot_mov',prop_x = 'time' ,add_horizontal_line=None,color = phi_r_color)
exp.plot_prop_movies('phi_lw_min_val','mean_wing',fig,case = 'plot_mov',prop_x = 'time' ,add_horizontal_line=None,color = phi_l_color)

exp.plot_prop_movies('phi_rw_max_val','mean_wing',fig,case = 'plot_mov',prop_x = 'time' ,add_horizontal_line=None,color = phi_r_color)
exp.plot_prop_movies('phi_lw_max_val','mean_wing',fig,case = 'plot_mov',prop_x = 'time' ,add_horizontal_line=None,color = phi_l_color)
fig.show()


In [ ]:
exp = experiments['manipulated_2023_08_09_60ms']#['manipulated_2022_02_03_dark']#['manipulated_2023_08_09_60ms']

prop = 'delta_ang_Xax'
fig = go.Figure()
exp.plot_prop_movies(prop,'body',fig,case = 'plot_mov',prop_x = 'time')
exp.plot_prop_movies('delta_ang_CM_dot_x','body',fig,case = 'plot_mov',prop_x = 'time')

fig.show()



phi_r_color = [[256,0,0]]*len(exp.mov_names)
phi_l_color = [[0,0,256]]*len(exp.mov_names)

fig = go.Figure()
exp.plot_prop_movies('phi_rw','wing',fig,case = 'plot_mov',prop_x = 'time',color = phi_r_color )
exp.plot_prop_movies('phi_lw','wing',fig,case = 'plot_mov',prop_x = 'time', color = phi_l_color)
fig.show()


Plot 3d trajectory

In [ ]:
mov = ['mov140','mov141','mov143','mov144','mov146','mov147']
exp = experiments['manipulated_2023_08_09_60ms']
exp.plot_3d_traj_movies('delta_ang_CM_dot_x',mov = mov)


In [ ]:


exp = experiments['manipulated_2023_08_09_60ms']
prop_name = 'delta_ang_CM_dot_x'
dict_v = {'lt_zero':[],'gt_zero':[]}
for mov_nm in exp.mov_names:
    mov = exp.get_mov(mov_nm)
    prop = mov.get_prop(prop_name,'body')
    if np.abs(np.nanmean(prop[:300]) - np.nanmean(prop[-300:]))>300:
        dict_v['lt_zero'].append(mov_nm )
    else:
        dict_v['gt_zero'].append(mov_nm )

fig = go.Figure()

exp.plot_prop_movies(prop_name,'body',fig,mov = dict_v['lt_zero'],case = 'plot_exp',prop_x = 'time',color = color_map[3],legend = f'cluster{1}')
exp.plot_prop_movies(prop_name,'body',fig,mov = dict_v['gt_zero'],case = 'plot_exp',prop_x = 'time',color = color_map[4],legend = f'cluster{2}')
fig.show()

prop_name = 'CM_dot_x_projected'
fig = go.Figure()

exp.plot_prop_movies(prop_name,'body',fig,mov = dict_v['lt_zero'],case = 'plot_exp',prop_x = 'time',color = color_map[3],legend = f'cluster{1}')
exp.plot_prop_movies(prop_name,'body',fig,mov = dict_v['gt_zero'],case = 'plot_exp',prop_x = 'time',color = color_map[4],legend = f'cluster{2}')
fig.show()



prop_name = 'slip_angle_beta'
fig = go.Figure()

exp.plot_prop_movies(prop_name,'body',fig,mov = dict_v['lt_zero'],case = 'plot_exp',prop_x = 'time',color = color_map[3],legend = f'cluster{1}')
exp.plot_prop_movies(prop_name,'body',fig,mov = dict_v['gt_zero'],case = 'plot_exp',prop_x = 'time',color = color_map[4],legend = f'cluster{2}')
fig.show()


prop_name = 'CM_dot_y_projected'
fig = go.Figure()

exp.plot_prop_movies(prop_name,'body',fig,mov = dict_v['lt_zero'],case = 'plot_exp',prop_x = 'time',color = color_map[3],legend = f'cluster{1}')
exp.plot_prop_movies(prop_name,'body',fig,mov = dict_v['gt_zero'],case = 'plot_exp',prop_x = 'time',color = color_map[4],legend = f'cluster{2}')
fig.show()


In [ ]:

def rotation_matrix(yaw,pitch,roll):
    roll_mat = np.vstack([[1,0,0],[0 ,np.cos(roll),-np.sin(roll)],[0, np.sin(roll), np.cos(roll)]])
    pitch_mat = np.vstack([[np.cos(pitch),0,np.sin(pitch)],[0, 1,0],[-np.sin(pitch), 0, np.cos(pitch)]])
    yaw_mat = np.vstack([[np.cos(yaw),-np.sin(yaw),0],[np.sin(yaw),np.cos(yaw),0],[0, 0, 1]])
    return yaw_mat @ pitch_mat @ roll_mat


for exp_name in experiments.keys():
    exp = experiments[exp_name]
    wing_body = 'body'
    for mov_name in exp.mov_names:
        mov = exp.get_mov(mov_name)

        yaw = mov.get_prop('yaw_body','body')[:,0]
        pitch = mov.get_prop('pitch_body','body')[:,0]
        roll = mov.get_prop('roll_body','body')[:,0]
        time = mov.get_prop('time','body')[:,0]

        rot_mat = [rotation_matrix(yaw*np.pi/180,-pitch*np.pi/180,roll*np.pi/180).T for yaw,pitch,roll in zip(yaw,pitch,roll)]
        vxyz = mov.get_prop('CM_real_x_body_dot_smth','body',three_col=3)
        axyz = mov.get_prop('CM_real_x_body_dot_dot_smth','body',three_col=3)

        uvw = np.vstack([rot_mat @ vxyz for rot_mat,vxyz in zip(rot_mat,vxyz)])
        uvw_dot = np.vstack([rot_mat @ axyz for rot_mat,axyz in zip(rot_mat,axyz)])

        vz_sp = uvw[:,0]*np.cos(np.radians(45)) + uvw[:,2]*np.cos(np.radians(45))
        vx_sp = uvw[:,0]*np.cos(np.radians(45)) - uvw[:,2]*np.cos(np.radians(45))



        az_sp = uvw_dot[:,0]*np.cos(np.radians(45)) + uvw_dot[:,2]*np.cos(np.radians(45))
        ax_sp = uvw_dot[:,0]*np.cos(np.radians(45)) - uvw_dot[:,2]*np.cos(np.radians(45))

        mov.data[wing_body] = np.hstack((mov.data[wing_body], uvw))
        mov.add_to_header(['u','v','w'],wing_body)

        
        mov.data[wing_body] = np.hstack((mov.data[wing_body], uvw_dot))
        mov.add_to_header(['u_dot','v_dot','w_dot'],wing_body)

        mov.data[wing_body] = np.vstack((mov.data[wing_body].T, vz_sp)).T
        mov.add_to_header(['vz_sp'],wing_body)
        mov.data[wing_body] = np.vstack((mov.data[wing_body].T, vx_sp)).T
        mov.add_to_header(['vx_sp'],wing_body)

        
        mov.data[wing_body] = np.vstack((mov.data[wing_body].T, az_sp)).T
        mov.add_to_header(['az_sp'],wing_body)
        mov.data[wing_body] = np.vstack((mov.data[wing_body].T, ax_sp)).T
        mov.add_to_header(['ax_sp'],wing_body)
# fig,ax = plt.subplots(2,1)
# ax[0].plot(time,vx_sp)
# ax[0].plot(time,uvw[:,1])
# ax[0].plot(time,vz_sp)

# ax[1].plot(vxyz)
    
# for mov_name in exp.mov_names:

    
    # mov.data[wing_body] = np.hstack((mov.data[wing_body], uvw))
    # mov.add_to_header(['u','v','w'],wing_body)

    # mov.data[wing_body] = np.vstack((mov.data[wing_body].T, asp_z)).T
    # mov.add_to_header(['asp_z'],wing_body)


In [ ]:
for name in ['ax_sp','az_sp','v_dot']:
    f,tm = exp.mean_time_series_prop(name,wing_body,window = (73*7)//2,dt = 1/16000)
    plt.plot(tm,f)

g:\Documents\experiment_analysis\data_analysis\Experiment.py:136: RuntimeWarning:

Mean of empty slice



Plot property and add interest point

In [ ]:

exp = experiments['manipulated_2023_08_06_40ms']
prop = 'CM_dot_dot_x_projected'

fig = go.Figure()
exp.plot_prop_movies(prop,'body',fig,case = 'plot_mov',prop_x = 'time')
exp.add_point_to_plot_movies('CM_dot_dot_x_projected_peaks_max',prop,fig) 
fig.show()



In [ ]:
fig = go.Figure()
exp = experiments['manipulated_2022_02_03_dark']
exp.plot_prop_movies('CM_dot_dot_y_projected','body',fig,case = 'plot_mov',prop_x = 'time')
fig.show()

phi_r_color = [[256,0,0]]*len(exp.mov_names)
phi_l_color = [[0,0,256]]*len(exp.mov_names)

fig = go.Figure()
exp = experiments['manipulated_2022_02_03_dark']
exp.plot_prop_movies('phi_rw','wing',fig,case = 'plot_mov',prop_x = 'time',color = phi_r_color )
exp.plot_prop_movies('phi_lw','wing',fig,case = 'plot_mov',prop_x = 'time', color = phi_l_color)

fig.show()

3D trajectory plot

In [ ]:

exp = experiments['manipulated_2023_08_09_60ms']
mov = ['mov60']
exp.plot_3d_traj_movies('CM_dot_x_projected',mov = mov)


Plot histograms of peaks, plot movies time seires data with the max/min/peak data point on them

In [ ]:
# Histograms
point_name = 'pitch_y_frame_dot_dot_peaks_max'
list_trace,layout = zip(*[exp.interest_point_hist(point_name,xbins = dict(start=0,end=300, size=5)) for exp in experiments.values()])
fig = go.Figure(data=list_trace, layout=layout[0]).show()

point_name = 'pitch_y_frame_max'
list_trace,layout = zip(*[exp.interest_point_hist(point_name,xbins = dict(start=0,end=300, size=5)) for exp in experiments.values()])
fig = go.Figure(data=list_trace, layout=layout[0]).show()


point_name = 'CM_dot_dot_projected_min'
list_trace,layout = zip(*[exp.interest_point_hist(point_name,xbins = dict(start=0,end=300, size=5)) for exp in experiments.values()])
fig = go.Figure(data=list_trace, layout=layout[0]).show()


point_name = 'amp_v_peaks_min'
list_trace,layout = zip(*[exp.interest_point_hist(point_name,xbins = dict(start=0,end=300, size=5)) for exp in experiments.values()])
fig = go.Figure(data=list_trace, layout=layout[0]).show()


point_name = 'CM_dot_projected_zero_v'
list_trace,layout = zip(*[exp.interest_point_hist(point_name,xbins = dict(start=0,end=300, size=5)) for exp in experiments.values()])
fig = go.Figure(data=list_trace, layout=layout[0]).show()

KeyError: 'CM_dot_dot_projected_min'

In [ ]:
exp = experiments['manipulated_2022_02_03_dark']
fig = go.Figure()
exp.plot_prop_movies('CM_dot_projected','body',fig,case = 'plot_mov',prop_x = 'time')
exp.add_point_to_plot_movies('amp_v_peaks_min','amp_v',fig) 
fig.show()

In [ ]:
# time series of movies with min/max on them
exp = experiments['manipulated_2022_02_03_dark']
fig = go.Figure()
exp.plot_prop_movies('pitch_y_frame_dot_dot','body',fig,case = 'plot_mov',prop_x = 'time')
exp.add_point_to_plot_movies('pitch_y_frame_dot_dot_peaks_max','pitch_y_frame_dot_dot',fig) 
fig.show()

fig = go.Figure()
exp.plot_prop_movies('CM_dot_projected','body',fig,case = 'plot_mov',prop_x = 'time')
exp.add_point_to_plot_movies('pitch_y_frame_dot_dot_peaks_max','CM_dot_projected',fig) 
fig.show()

In [ ]:

fig = go.Figure()
exp.plot_prop_movies('roll_x_frame','body',fig,case = 'plot_mov',prop_x = 'time')
# exp.add_point_to_plot_movies('CM_dot_projected_zero_v','CM_dot_y_projected',fig) 
exp.plot_prop_movies('yaw_z_frame','body',fig,case = 'plot_mov',prop_x = 'time',yaxis = 'y2')
# exp.add_point_to_plot_movies('CM_dot_projected_zero_v','roll_body',fig,yaxis = 'y2') 

# yax_dict[f"yaxis2"] = (dict(title=prop,titlefont=dict(color=color),tickfont=dict(color=color),side = side[i],position=position[i]))



fig.update_layout(title = exp.experiment_name,
    xaxis=dict(domain=[0, 0.95]),
    yaxis=dict(title='roll_x_frame', titlefont=dict(color='red'), tickfont=dict(color='red'), side='left', position=0),
    yaxis2=dict(title='yaw_y_frame', titlefont=dict(color='green'), tickfont=dict(color='green'), side='right', position=0.95),
)

fig.show()

Plot all experiments - time series

In [ ]:
fig = go.Figure()
prop = 'pitch_y_frame_dot_dot'
[exp.plot_prop_movies(prop,'body',fig,case = 'plot_exp',prop_x = 'time',color = color_map[idx],legend = exp.pertubation_name) for idx,exp in enumerate(experiments.values())]
fig.show()

Calculate and plot mean property (time series,  mean per frame of all movies)

In [ ]:
from scipy import stats


exp = experiments[list(experiments.keys())[0]]
prop = 'pitch_y_frame'
prop = 'r'
prop = 'CM_dot_dot_projected'
props = ['delta_ang','delta_ang']

window = (73*3)//2
fig,ax = plt.subplots(2,1,sharex=True)
max_prop = {}
for idx,prop in enumerate(props):
    max_prop[prop] = [] 
    for exp in experiments.values():
        mov_to_keep = []
        acc_mean,time_inter = exp.mean_time_series_prop(prop,'vectors')
        time_to_check = (time_inter > 0 ) & (time_inter <100 )
        time_for_max = time_inter[time_to_check]
        max_prop[prop].append(time_for_max[np.nanargmax(acc_mean[time_to_check])])
        ax[idx].plot(time_inter,acc_mean, label = exp.experiment_name)


plt.legend()

g:\Documents\experiment_analysis\data_analysis\Experiment.py:121: RuntimeWarning:

Mean of empty slice



In [16]:
mov.header['mean_body']

{'pitch_body': 0,
 'yaw_body': 1,
 'roll_body': 2,
 'pitch_body_dot': 3,
 'yaw_body_dot': 4,
 'roll_body_dot': 5,
 'pitch_body_dot_dot': 6,
 'yaw_body_dot_dot': 7,
 'roll_body_dot_dot': 8,
 'frames': 9,
 'time': 10,
 'CM_real_x_body': 11,
 'CM_real_y_body': 12,
 'CM_real_z_body': 13,
 'phi_rw_max_idx': 14,
 'phi_rw_max_val': 15,
 'phi_lw_max_idx': 16,
 'phi_lw_max_val': 17,
 'phi_rw_min_idx': 18,
 'phi_rw_min_val': 19,
 'phi_lw_min_idx': 20,
 'phi_lw_min_val': 21,
 'yaw_z_frame': 22,
 'pitch_y_frame': 23,
 'roll_x_frame': 24,
 'mean_idx': 25,
 'freq_phi_rw_min_idx': 26,
 'CM_dot_dot_x_projected': 27,
 'delta_ang_CM_dot_x': 28,
 'delta_ang_Xax': 29}

Try PCA

In [27]:
exp.mov_names

In [57]:
from sklearn.decomposition import PCA
import pandas as pd
from sklearn.cluster import KMeans

mov_feat = []
props = ['CM_dot_dot_x_projected','freq_phi_rw_min_idx']

exp_pd = []
for exp in list(experiments.values())[1:2]:
    mov_feat = []
    for idx,mov_name in enumerate(exp.mov_names):
        mov = exp.get_mov(mov_name)
        mov_feat.append(mov.data['mean_body'])

    featpd_exp = pd.DataFrame(data = np.vstack(mov_feat),columns = mov.header['mean_body'])
    featpd_exp['exp_name'] = exp.experiment_name
    # featpd_exp['mov'] = exp.mov_names
    exp_pd.append(featpd_exp)

feat = pd.concat(exp_pd,axis = 0).dropna().reset_index()


dataforpca = feat[props]
n_comp = len(props)
pca = PCA(n_components=n_comp)
principalComponents = pca.fit_transform(dataforpca)
principalDf = pd.DataFrame(data = principalComponents
             , columns = [f'pca_{idx}' for idx in range(n_comp)])

pca_name = [f'pca_{idx}' for idx in range(pca.n_components_)]


featpd = pd.concat([feat, principalDf], axis = 1)
kmeans = KMeans(n_clusters=n_comp, random_state=0, n_init="auto").fit(featpd[pca_name])
print(kmeans.labels_)
pca.components_


[0 0 0 ... 0 0 0]


array([[-9.89891583e-04,  9.99999510e-01],
       [-9.99999510e-01, -9.89891583e-04]])

In [58]:
for lbl in np.unique(kmeans.labels_):
    feat.iloc[kmeans.labels_==lbl]['delta_ang_CM_dot_x'].plot(marker = '*')

In [31]:

fig = go.Figure()

for lbl in np.unique(kmeans.labels_):
    exp_names = np.unique(feat.iloc[kmeans.labels_==lbl]['exp_name'])
    feat_lbl = feat.iloc[kmeans.labels_==lbl]
    for idx,exp_name in enumerate(exp_names): 
        mov_exp_lbl = list(feat_lbl.loc[feat_lbl['exp_name'] == exp_name])

        prop = 'pitch_y_frame_dot'
        prop = 'CM_dot_dot_projected'
        # prop = 'pitch_body'

        color_map = colormap.datad["tab10"]['listed']

        experiments[exp_name].plot_prop_movies(prop,'body',fig,mov = mov_exp_lbl,case = 'plot_exp',prop_x = 'time',color = color_map[lbl],legend = f'{exp_name}_cluster{lbl}')
fig.show()
# feat.iloc[kmeans.labels_==lbl]['exp_name'] == exp_names[0]
from scipy import stats


exp = experiments[list(experiments.keys())[0]]
prop = 'pitch_y_frame'
prop = 'q'
prop = 'CM_dot_dot_projected'
props = ['pitch_y_frame_dot_dot','pitch_y_frame_dot']

window = (73*3)//2
fig,ax = plt.subplots(2,1,sharex=True)
max_prop = {}
for idx,prop in enumerate(props):
    max_prop[prop] = [] 
    for exp in experiments.values():
        mov_to_keep = []
        acc_mean,time_inter = exp.mean_time_series_prop(prop,'body')
        time_to_check = (time_inter > 0 ) & (time_inter <100 )
        time_for_max = time_inter[time_to_check]
        max_prop[prop].append(time_for_max[np.nanargmax(acc_mean[time_to_check])])
        ax[idx].plot(time_inter,acc_mean)

KeyError: 'index'

Plot and save figures

In [ ]:
exp

In [ ]:
import plotly.graph_objects as go
import plotly
import matplotlib.cm as colormap
color_map = colormap.datad["tab10"]['listed']



def plot_and_save_movie_prop(exp,yprop,wing_body,title,add_horizontal_line = None):
    
    fig = go.Figure()

    exp.plot_prop_movies(yprop,wing_body,fig,case = 'plot_mov',prop_x = 'time',add_horizontal_line = add_horizontal_line)
    fig.update_layout(title = title)
   
    fig.show()
    plotly.offline.plot(fig, filename=f'{exp.figures_path}/{yprop}_plot.html',auto_open=False)




for exp in experiments:
    try:
        exp = experiments[exp]
        exp.plot_3d_traj_movies('CM_dot_dot_x_projected',save_plot = True, mov = False)

        plot_and_save_movie_prop(exp, 'slip_angle_beta', 'vectors','dot product between Xbody_xy and V_xy', add_horizontal_line = 0)
        plot_and_save_movie_prop(exp, 'delta_ang_CM_dot_x','vectors','dot product between initial V_xy and V_xy',add_horizontal_line = None)
        plot_and_save_movie_prop(exp, 'delta_ang_Xax','vectors','dot product between initial Xbody_xy and Xbody_xy',add_horizontal_line = None)

        plot_and_save_movie_prop(exp, 'CM_dot_dot_x_projected','acceleration projected on XY',wing_body = 'body',add_horizontal_line = 0)
        plot_and_save_movie_prop(exp, 'pitch_y_frame','pitch angle rotated to first frame', 'body',add_horizontal_line = 0)
        plot_and_save_movie_prop(exp, 'pitch_y_frame_dot','pitch_dot rotated to first frame' 'body',add_horizontal_line = 0)
        plot_and_save_movie_prop(exp, 'CM_dot_x_projected','velocity projected on XY', 'body',add_horizontal_line = 0)
    except:
        continue
    # exp.mean_mean_props_movies('phi_rw_min_val','phi_lw_min_val','mean_wing','mean_front_phi')
    # exp.mean_mean_props_movies('phi_rw','phi_lw','mean_wing','mean_mean_phi')

    # plot_and_save_movie_prop(exp,yprop = 'mean_front_phi',wing_body = 'mean_wing')
    # plot_and_save_movie_prop(exp,yprop = 'mean_mean_phi',wing_body = 'mean_wing')

